In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework
import pandas as pd

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()


In [3]:
workspace_name="helper-demo-workspace-2"
pipeline_name="helper-demo-pipeline-1"
model_name="helper-demo-model-1"
model_file_name="./models/rf_model.onnx"

## get_workspace

In [4]:
wl.search_model_versions("helper-demo-model-1")

[]

In [5]:
wl.search_model_versions("helper-demo-model-1", workspace_id=26)

[]

In [6]:
wl.search_model_versions("helper-demo-model-1", workspace_name="helper-demo-workspace-1")

[]

### Error when workspace does not exist

In [7]:
workspace = wl.get_workspace(name=workspace_name)

Exception: Error: Workspace with name helper-demo-workspace-2 does not exist. If you would like to create one, send in the request with `create_if_not_exist` flag set to True.

### Create new workspace with `create_if_not_exist` flag

In [8]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
workspace

{'name': 'helper-demo-workspace-2', 'id': 9, 'archived': False, 'created_by': 'db8299d1-ee3b-4ca1-bda1-9e616c5dad82', 'created_at': '2024-07-29T15:38:18.06432+00:00', 'models': [], 'pipelines': []}

In [9]:
wl.set_current_workspace(workspace)

{'name': 'helper-demo-workspace-2', 'id': 9, 'archived': False, 'created_by': 'db8299d1-ee3b-4ca1-bda1-9e616c5dad82', 'created_at': '2024-07-29T15:38:18.06432+00:00', 'models': [], 'pipelines': []}

### Return existing workspace

In [10]:
workspace = wl.get_workspace(name=workspace_name)
workspace

{'name': 'helper-demo-workspace-2', 'id': 9, 'archived': False, 'created_by': 'db8299d1-ee3b-4ca1-bda1-9e616c5dad82', 'created_at': '2024-07-29T15:38:18.06432+00:00', 'models': [], 'pipelines': []}

## create model and pipeline

In [11]:
helper_model = (wl.upload_model(model_name, 
                                        model_file_name, 
                                        framework=Framework.ONNX)
                                        .configure(tensor_fields=["tensor"])
                        )

In [12]:
helper_model.version()

'f3db1bee-b29f-4852-bcba-ac47a7ce6e6b'

## get_model

### Error when model not found

In [13]:
wl.get_model(name="no-such-model")

Exception: Error: A model with the name no-such-model does not exist.

In [14]:
model = wl.get_model(name=model_name)
display(model)

Name,helper-demo-model-1
Version,f3db1bee-b29f-4852-bcba-ac47a7ce6e6b
File Name,rf_model.onnx
SHA,e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-29-Jul 15:38:22
Workspace id,9
Workspace name,helper-demo-workspace-2


In [15]:
model_version = wl.get_model(name=model_name).version()

In [16]:
wl.get_model(name=model_name, version="no-such-version")

Exception: Error: A model with the version no-such-version not found in this workspace.

In [17]:
model_1 = wl.get_model(name=model_name, version=model_version)
model_1

Name,helper-demo-model-1
Version,f3db1bee-b29f-4852-bcba-ac47a7ce6e6b
File Name,rf_model.onnx
SHA,e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-29-Jul 15:38:22
Workspace id,9
Workspace name,helper-demo-workspace-2


## get_pipeline

In [18]:
pipeline = wl.build_pipeline(pipeline_name)
pipeline

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:38:30.152051+00:00
deployed,(none)
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,None
accel,None
tags,
versions,084330f4-f19c-473a-ac00-c334ad08c9df
steps,


### Error when pipeline doesn't exist

In [19]:
wl.get_pipeline(name="no-such-pipeline")

Exception: Pipeline no-such-pipeline not found in this workspace.

In [20]:
wl.get_pipeline(name=pipeline_name)

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:38:30.152051+00:00
deployed,(none)
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,None
accel,None
tags,
versions,084330f4-f19c-473a-ac00-c334ad08c9df
steps,


In [24]:
pipeline_version = wl.get_pipeline(name=pipeline_name).versions()[0]
pipeline_version

name,helper-demo-pipeline-1
version,084330f4-f19c-473a-ac00-c334ad08c9df
creation_time,2024-29-Jul 15:38:30
last_updated_time,2024-29-Jul 15:38:30
deployed,False
tags,
steps,


In [33]:
pipeline_version_id = pipeline_version._name
pipeline_version_id

'084330f4-f19c-473a-ac00-c334ad08c9df'

### Error when version doesn't exist

In [22]:
wl.get_pipeline(name=pipeline_name, version="no-such-version")

Exception: Pipeline version no-such-version not found in this workspace.

In [34]:
wl.get_pipeline(name=pipeline_name, version=pipeline_version_id)

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:38:30.152051+00:00
deployed,(none)
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,None
accel,None
tags,
versions,084330f4-f19c-473a-ac00-c334ad08c9df
steps,


### use returned model and pipeline from get_model / get_pipeline

In [35]:
mainpipeline = wl.get_pipeline(name=pipeline_name)
mainpipeline.clear()

model_1 = wl.get_model(name=model_name)
mainpipeline.add_model_step(model_1)
mainpipeline

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:38:30.152051+00:00
deployed,(none)
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,None
accel,None
tags,
versions,084330f4-f19c-473a-ac00-c334ad08c9df
steps,


In [38]:
deploy_config = wallaroo.DeploymentConfigBuilder().cpus(0.25).memory("1Gi").build()
mainpipeline.deploy(deployment_config = deploy_config)

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:46:51.751491+00:00
deployed,True
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,x86
accel,none
tags,
versions,"72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df"
steps,helper-demo-model-1


In [37]:
mainpipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.1.4',
   'name': 'engine-568bd5fb44-529f2',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'helper-demo-pipeline-1',
      'status': 'Running',
      'version': '29ca55a0-da90-4ec5-9ca6-01f3583d0f55'}]},
   'model_statuses': {'models': [{'name': 'helper-demo-model-1',
      'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6',
      'status': 'Running',
      'version': 'f3db1bee-b29f-4852-bcba-ac47a7ce6e6b'}]}}],
 'engine_lbs': [{'ip': '10.28.1.2',
   'name': 'engine-lb-6b59985857-cps5w',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

In [39]:
normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})
result = mainpipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,anomaly.count
0,2024-07-29 15:46:57.732,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0,...",[718013.7],0


In [40]:
mainpipeline.versions()

In [41]:
mainpipeline.undeploy()

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:46:51.751491+00:00
deployed,False
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,x86
accel,none
tags,
versions,"72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df"
steps,helper-demo-model-1


### Get existing pipeline version and redeploy that specific version

#### Creating new versions

In [42]:
new_model = (wl.upload_model("helper-model-replace", 
                             "./models/gbr_model.onnx", 
                             framework=Framework.ONNX)
                             .configure(tensor_fields=["tensor"])
                        )
new_model

Name,helper-model-replace
Version,42b85a68-2a2e-46e5-9b4c-76415ce15d07
File Name,gbr_model.onnx
SHA,ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-29-Jul 15:47:39
Workspace id,9
Workspace name,helper-demo-workspace-2


In [43]:
pipeline.clear()
pipeline.add_model_step(new_model)
pipeline.steps()

[{'ModelInference': {'models': [{'name': 'helper-model-replace', 'version': '42b85a68-2a2e-46e5-9b4c-76415ce15d07', 'sha': 'ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a'}]}}]

In [44]:
deploy_config = wallaroo.DeploymentConfigBuilder().cpus(0.25).memory("1Gi").build()
pipeline.deploy(deployment_config = deploy_config)

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:47:40.501743+00:00
deployed,True
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,x86
accel,none
tags,
versions,"ed74510b-3c51-4b24-abb7-71ee9f069f2a, 72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df"
steps,helper-model-replace


In [45]:
mainpipeline.undeploy()

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:46:51.751491+00:00
deployed,False
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,x86
accel,none
tags,
versions,"72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df"
steps,helper-model-replace


## Get specific version and redeploy

In [46]:
workspace

{'name': 'helper-demo-workspace-2', 'id': 9, 'archived': False, 'created_by': 'db8299d1-ee3b-4ca1-bda1-9e616c5dad82', 'created_at': '2024-07-29T15:38:18.06432+00:00', 'models': [{'name': 'helper-demo-model-1', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 7, 29, 15, 38, 22, 277189, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 7, 29, 15, 38, 22, 277189, tzinfo=tzutc())}, {'name': 'helper-model-replace', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 7, 29, 15, 47, 39, 533644, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 7, 29, 15, 47, 39, 533644, tzinfo=tzutc())}], 'pipelines': [{'name': 'helper-demo-pipeline-1', 'create_time': datetime.datetime(2024, 7, 29, 15, 38, 30, 152051, tzinfo=tzutc()), 'definition': '[]'}]}

In [47]:
wl.search_pipelines(search_term=pipeline_name, workspace_name=workspace_name)

name,version,creation_time,last_updated_time,deployed,workspace_id,workspace_name,tags,steps
helper-demo-pipeline-1,ed74510b-3c51-4b24-abb7-71ee9f069f2a,2024-29-Jul 15:47:40,2024-29-Jul 15:47:40,False,9,helper-demo-workspace-2,,helper-model-replace
helper-demo-pipeline-1,72b907a1-74ee-43ad-9e37-f3edf3a0f7fe,2024-29-Jul 15:46:51,2024-29-Jul 15:46:51,(unknown),9,helper-demo-workspace-2,,helper-demo-model-1
helper-demo-pipeline-1,29ca55a0-da90-4ec5-9ca6-01f3583d0f55,2024-29-Jul 15:40:25,2024-29-Jul 15:40:25,(unknown),9,helper-demo-workspace-2,,helper-demo-model-1
helper-demo-pipeline-1,084330f4-f19c-473a-ac00-c334ad08c9df,2024-29-Jul 15:38:30,2024-29-Jul 15:38:30,(unknown),9,helper-demo-workspace-2,,


In [48]:
wl.search_pipelines(search_term=pipeline_name, workspace_id=9)

name,version,creation_time,last_updated_time,deployed,workspace_id,workspace_name,tags,steps
helper-demo-pipeline-1,ed74510b-3c51-4b24-abb7-71ee9f069f2a,2024-29-Jul 15:47:40,2024-29-Jul 15:47:40,False,9,helper-demo-workspace-2,,helper-model-replace
helper-demo-pipeline-1,72b907a1-74ee-43ad-9e37-f3edf3a0f7fe,2024-29-Jul 15:46:51,2024-29-Jul 15:46:51,(unknown),9,helper-demo-workspace-2,,helper-demo-model-1
helper-demo-pipeline-1,29ca55a0-da90-4ec5-9ca6-01f3583d0f55,2024-29-Jul 15:40:25,2024-29-Jul 15:40:25,(unknown),9,helper-demo-workspace-2,,helper-demo-model-1
helper-demo-pipeline-1,084330f4-f19c-473a-ac00-c334ad08c9df,2024-29-Jul 15:38:30,2024-29-Jul 15:38:30,(unknown),9,helper-demo-workspace-2,,


In [49]:
wl.search_pipelines()

name,version,creation_time,last_updated_time,deployed,workspace_id,workspace_name,tags,steps
helper-demo-pipeline-1,ed74510b-3c51-4b24-abb7-71ee9f069f2a,2024-29-Jul 15:47:40,2024-29-Jul 15:47:40,False,9,helper-demo-workspace-2,,helper-model-replace
helper-demo-pipeline-1,72b907a1-74ee-43ad-9e37-f3edf3a0f7fe,2024-29-Jul 15:46:51,2024-29-Jul 15:46:51,(unknown),9,helper-demo-workspace-2,,helper-demo-model-1
helper-demo-pipeline-1,29ca55a0-da90-4ec5-9ca6-01f3583d0f55,2024-29-Jul 15:40:25,2024-29-Jul 15:40:25,(unknown),9,helper-demo-workspace-2,,helper-demo-model-1
helper-demo-pipeline-1,084330f4-f19c-473a-ac00-c334ad08c9df,2024-29-Jul 15:38:30,2024-29-Jul 15:38:30,(unknown),9,helper-demo-workspace-2,,
tagtestpipeline,b95151d8-35b6-47fc-9a3e-8cb2b8005687,2024-26-Jul 17:54:13,2024-26-Jul 17:54:13,(unknown),8,tagtestworkspace,,
tagtestpipeline,98e48177-2c95-4ea4-a996-5747d9aa7d7b,2024-26-Jul 17:51:57,2024-26-Jul 17:51:57,(unknown),8,tagtestworkspace,,
tagtestpipeline,2c4db874-0f07-4e30-82ad-43cf1d4073a0,2024-26-Jul 17:49:50,2024-26-Jul 17:49:50,(unknown),8,tagtestworkspace,,
housing-pipe,ac0f3d0c-4b7b-4249-a72f-faaa58b393ea,2024-26-Jul 16:11:39,2024-26-Jul 16:11:39,False,7,housepricing,,"housepricemodel, preprocess-step, postprocess-step"
housing-pipe,d4dc06b3-f28d-454a-be72-9d12dd30c6c2,2024-26-Jul 16:09:32,2024-26-Jul 16:09:32,(unknown),7,housepricing,,"preprocess-step, housepricemodel, postprocess-step"
housing-pipe,d610c4f2-5c05-41d6-b727-36c2dc21a04a,2024-26-Jul 16:03:54,2024-26-Jul 16:03:54,(unknown),7,housepricing,,"preprocess-step, housepricemodel, postprocess-step"


pipelines_by_name

In [50]:
wl.pipelines_by_name(pipeline_name=pipeline_name)

name,created,last_updated,deployed,workspace_id,workspace_name,arch,accel,tags,versions,steps,published
helper-demo-pipeline-1,2024-29-Jul 15:38:30,2024-29-Jul 15:47:40,False,9,helper-demo-workspace-2,x86,none,,"ed74510b-3c51-4b24-abb7-71ee9f069f2a, 72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df",helper-model-replace,False


In [51]:
wl.pipelines_by_name(pipeline_name=pipeline_name, workspace_id=9)

name,created,last_updated,deployed,workspace_id,workspace_name,arch,accel,tags,versions,steps,published
helper-demo-pipeline-1,2024-29-Jul 15:38:30,2024-29-Jul 15:47:40,False,9,helper-demo-workspace-2,x86,none,,"ed74510b-3c51-4b24-abb7-71ee9f069f2a, 72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df",helper-model-replace,False


In [52]:
wl.pipelines_by_name(pipeline_name=pipeline_name, workspace_name=workspace_name)

name,created,last_updated,deployed,workspace_id,workspace_name,arch,accel,tags,versions,steps,published
helper-demo-pipeline-1,2024-29-Jul 15:38:30,2024-29-Jul 15:47:40,False,9,helper-demo-workspace-2,x86,none,,"ed74510b-3c51-4b24-abb7-71ee9f069f2a, 72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df",helper-model-replace,False


In [56]:
pipeline_version_id

'084330f4-f19c-473a-ac00-c334ad08c9df'

In [65]:
pipeline_with_version_set = wl.get_pipeline(name=pipeline_name, version="29ca55a0-da90-4ec5-9ca6-01f3583d0f55")
pipeline_with_version_set

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:47:40.501743+00:00
deployed,False
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,x86
accel,none
tags,
versions,"ed74510b-3c51-4b24-abb7-71ee9f069f2a, 72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df"
steps,helper-model-replace


In [67]:
pipeline_with_version_set.add_model_step(new_model)

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:47:40.501743+00:00
deployed,False
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,x86
accel,none
tags,
versions,"ed74510b-3c51-4b24-abb7-71ee9f069f2a, 72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df"
steps,helper-model-replace


In [68]:
deploy_config = wallaroo.DeploymentConfigBuilder().cpus(0.25).memory("1Gi").build()
pipeline_with_version_set.deploy(deployment_config = deploy_config)

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:54:36.177376+00:00
deployed,True
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,x86
accel,none
tags,
versions,"37c3d9ec-93aa-4b69-9092-3f49fb496a78, ed74510b-3c51-4b24-abb7-71ee9f069f2a, 72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df"
steps,helper-model-replace


In [69]:
pipeline_with_version_set.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.1.10',
   'name': 'engine-f56cc86c-2f6kh',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'helper-demo-pipeline-1',
      'status': 'Running',
      'version': '37c3d9ec-93aa-4b69-9092-3f49fb496a78'}]},
   'model_statuses': {'models': [{'name': 'helper-model-replace',
      'sha': 'ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a',
      'status': 'Running',
      'version': '42b85a68-2a2e-46e5-9b4c-76415ce15d07'}]}}],
 'engine_lbs': [{'ip': '10.28.1.9',
   'name': 'engine-lb-6b59985857-d6d8m',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

In [70]:
pipeline_with_version_set.undeploy()

name,helper-demo-pipeline-1
created,2024-07-29 15:38:30.152051+00:00
last_updated,2024-07-29 15:54:36.177376+00:00
deployed,False
workspace_id,9
workspace_name,helper-demo-workspace-2
arch,x86
accel,none
tags,
versions,"37c3d9ec-93aa-4b69-9092-3f49fb496a78, ed74510b-3c51-4b24-abb7-71ee9f069f2a, 72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df"
steps,helper-model-replace


In [71]:
wl.list_pipelines()

name,created,last_updated,deployed,workspace_id,workspace_name,arch,accel,tags,versions,steps,published
housing-pipe,2024-26-Jul 16:03:54,2024-26-Jul 16:11:39,False,7,housepricing,x86,none,,"ac0f3d0c-4b7b-4249-a72f-faaa58b393ea, d4dc06b3-f28d-454a-be72-9d12dd30c6c2, d610c4f2-5c05-41d6-b727-36c2dc21a04a, 5e5547d8-a722-4000-9aa8-ceca8eee589e",housepricemodel,False
tagtestpipeline,2024-26-Jul 17:49:50,2024-26-Jul 17:54:13,(unknown),8,tagtestworkspace,None,None,,"b95151d8-35b6-47fc-9a3e-8cb2b8005687, 98e48177-2c95-4ea4-a996-5747d9aa7d7b, 2c4db874-0f07-4e30-82ad-43cf1d4073a0",,False
helper-demo-pipeline-1,2024-29-Jul 15:38:30,2024-29-Jul 15:54:36,False,9,helper-demo-workspace-2,x86,none,,"37c3d9ec-93aa-4b69-9092-3f49fb496a78, ed74510b-3c51-4b24-abb7-71ee9f069f2a, 72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df",helper-model-replace,False


In [72]:
wl.list_pipelines(workspace_id=9)

name,created,last_updated,deployed,workspace_id,workspace_name,arch,accel,tags,versions,steps,published
helper-demo-pipeline-1,2024-29-Jul 15:38:30,2024-29-Jul 15:54:36,False,9,helper-demo-workspace-2,x86,none,,"37c3d9ec-93aa-4b69-9092-3f49fb496a78, ed74510b-3c51-4b24-abb7-71ee9f069f2a, 72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df",helper-model-replace,False


In [73]:
wl.list_pipelines(workspace_name=workspace_name)

name,created,last_updated,deployed,workspace_id,workspace_name,arch,accel,tags,versions,steps,published
helper-demo-pipeline-1,2024-29-Jul 15:38:30,2024-29-Jul 15:54:36,False,9,helper-demo-workspace-2,x86,none,,"37c3d9ec-93aa-4b69-9092-3f49fb496a78, ed74510b-3c51-4b24-abb7-71ee9f069f2a, 72b907a1-74ee-43ad-9e37-f3edf3a0f7fe, 29ca55a0-da90-4ec5-9ca6-01f3583d0f55, 084330f4-f19c-473a-ac00-c334ad08c9df",helper-model-replace,False


In [74]:
wl.search_models(search_term=model_name)

name,version,file_name,image_path,arch,accel,last_update_time,workspace_id,workspace_name
helper-demo-model-1,f3db1bee-b29f-4852-bcba-ac47a7ce6e6b,rf_model.onnx,None,x86,none,2024-07-29 15:38:22.277189+00:00,9,helper-demo-workspace-2


In [75]:
wl.search_models(search_term=model_name, workspace_name=workspace_name)

name,version,file_name,image_path,arch,accel,last_update_time,workspace_id,workspace_name
helper-demo-model-1,f3db1bee-b29f-4852-bcba-ac47a7ce6e6b,rf_model.onnx,None,x86,none,2024-07-29 15:38:22.277189+00:00,9,helper-demo-workspace-2


In [76]:
wl.search_models(search_term=model_name, workspace_id=9)

name,version,file_name,image_path,arch,accel,last_update_time,workspace_id,workspace_name
helper-demo-model-1,f3db1bee-b29f-4852-bcba-ac47a7ce6e6b,rf_model.onnx,None,x86,none,2024-07-29 15:38:22.277189+00:00,9,helper-demo-workspace-2


Search by model name and version.

In [78]:
wl.model_by_name(name=model_name, version="f3db1bee-b29f-4852-bcba-ac47a7ce6e6b")

Name,helper-demo-model-1
Version,f3db1bee-b29f-4852-bcba-ac47a7ce6e6b
File Name,rf_model.onnx
SHA,e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-29-Jul 15:38:22
Workspace id,9
Workspace name,helper-demo-workspace-2


Search by model name and version filtered by workspace id.

In [79]:
wl.model_by_name(name=model_name, 
                 version="f3db1bee-b29f-4852-bcba-ac47a7ce6e6b",
                 workspace_id=9)

Name,helper-demo-model-1
Version,f3db1bee-b29f-4852-bcba-ac47a7ce6e6b
File Name,rf_model.onnx
SHA,e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-29-Jul 15:38:22
Workspace id,9
Workspace name,helper-demo-workspace-2


Search by model name and version filtered by workspace name.

In [80]:
wl.model_by_name(name=model_name, 
                 version="f3db1bee-b29f-4852-bcba-ac47a7ce6e6b",
                 workspace_name=workspace_name)

Name,helper-demo-model-1
Version,f3db1bee-b29f-4852-bcba-ac47a7ce6e6b
File Name,rf_model.onnx
SHA,e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-29-Jul 15:38:22
Workspace id,9
Workspace name,helper-demo-workspace-2


## List Models

List models in the current workspace.

In [81]:
wl.list_models()

Name,# of Versions,Owner ID,Last Updated,Created At,Workspace id,Workspace name
helper-demo-model-1,1,"""""",2024-07-29 15:38:22.277189+00:00,2024-07-29 15:38:22.277189+00:00,9,helper-demo-workspace-2
helper-model-replace,1,"""""",2024-07-29 15:47:39.533644+00:00,2024-07-29 15:47:39.533644+00:00,9,helper-demo-workspace-2


List models across workspaces filtered by workspace id.

In [82]:
wl.list_models(workspace_id=9)

Name,# of Versions,Owner ID,Last Updated,Created At,Workspace id,Workspace name
helper-demo-model-1,1,"""""",2024-07-29 15:38:22.277189+00:00,2024-07-29 15:38:22.277189+00:00,9,helper-demo-workspace-2
helper-model-replace,1,"""""",2024-07-29 15:47:39.533644+00:00,2024-07-29 15:47:39.533644+00:00,9,helper-demo-workspace-2


List models across worksapces filtered by workspace name.

In [83]:
wl.list_models(workspace_name=workspace_name)

Name,# of Versions,Owner ID,Last Updated,Created At,Workspace id,Workspace name
helper-demo-model-1,1,"""""",2024-07-29 15:38:22.277189+00:00,2024-07-29 15:38:22.277189+00:00,9,helper-demo-workspace-2
helper-model-replace,1,"""""",2024-07-29 15:47:39.533644+00:00,2024-07-29 15:47:39.533644+00:00,9,helper-demo-workspace-2
